In [18]:
import pandas as pd
import numpy as np
from collections import defaultdict
from collections import deque




In [19]:
path = "./adult.csv"

df = pd.read_csv(path)

df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [ ]:

print(df.dtypes)

numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
print("Numeric Columns:", numeric_cols)

def generalize_numeric_columns(df):
    df = df.copy()
    
    for col in df.select_dtypes(include=['int64', 'float64']).columns:
        if 'age' in col.lower():
            df[col] = pd.cut(df[col], bins=[0, 20, 30, 40, 50, 60, 100],
                                        labels=["0-20", "21-30", "31-40", "41-50", "51-60", "60+"])
        
        elif 'income' in col.lower() or 'salary' in col.lower():
            df[col] = pd.cut(df[col], bins=[0, 20000, 40000, 60000, 80000, 100000, 9999999],
                                          labels=["<20K", "20-40K", "40-60K", "60-80K", "80-100K", "100K+"])
        
        elif 'hour' in col.lower():  # e.g. hours_per_week
            df[col + '_range'] = pd.cut(df[col], bins=[0, 20, 40, 60, 80, 100],
                                        labels=["0-20", "21-40", "41-60", "61-80", "81-100"])
        
        else:
            # Generic fallback binning for other numeric columns
            df[col + '_binned'] = pd.qcut(df[col], q=4, duplicates='drop')

    return df




age                 int64
workclass          object
fnlwgt              int64
education          object
educational-num     int64
marital-status     object
occupation         object
relationship       object
race               object
gender             object
capital-gain        int64
capital-loss        int64
hours-per-week      int64
native-country     object
income             object
dtype: object
Numeric Columns: ['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss', 'hours-per-week']


In [24]:
df_generalized = generalize_numeric_columns(df)
df_generalized

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income,fnlwgt_binned,educational-num_binned,capital-gain_binned,capital-loss_binned,hours-per-week_range
0,21-30,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K,"(178144.5, 237642.0]","(0.999, 9.0]","(-0.001, 99999.0]","(-0.001, 4356.0]",21-40
1,31-40,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K,"(12284.999, 117550.5]","(0.999, 9.0]","(-0.001, 99999.0]","(-0.001, 4356.0]",41-60
2,21-30,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K,"(237642.0, 1490400.0]","(10.0, 12.0]","(-0.001, 99999.0]","(-0.001, 4356.0]",21-40
3,41-50,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K,"(117550.5, 178144.5]","(9.0, 10.0]","(-0.001, 99999.0]","(-0.001, 4356.0]",21-40
4,0-20,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K,"(12284.999, 117550.5]","(9.0, 10.0]","(-0.001, 99999.0]","(-0.001, 4356.0]",21-40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,21-30,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K,"(237642.0, 1490400.0]","(10.0, 12.0]","(-0.001, 99999.0]","(-0.001, 4356.0]",21-40
48838,31-40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K,"(117550.5, 178144.5]","(0.999, 9.0]","(-0.001, 99999.0]","(-0.001, 4356.0]",21-40
48839,51-60,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K,"(117550.5, 178144.5]","(0.999, 9.0]","(-0.001, 99999.0]","(-0.001, 4356.0]",21-40
48840,21-30,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K,"(178144.5, 237642.0]","(0.999, 9.0]","(-0.001, 99999.0]","(-0.001, 4356.0]",0-20


In [ ]:
# import pandas as pd
# import numpy as np

def mean_square_contingency(df, col1, col2):
    contingency = pd.crosstab(df[col1], df[col2])
    total = contingency.to_numpy().sum()

    f_i = contingency.sum(axis=1).to_numpy()  # row sums
    f_j = contingency.sum(axis=0).to_numpy()  # col sums

    phi2 = 0
    for i in range(contingency.shape[0]):
        for j in range(contingency.shape[1]):
            expected = f_i[i] * f_j[j] / total
            observed = contingency.iat[i, j]
            if expected > 0:
                phi2 += ((observed - expected) ** 2) / expected

    min_d = min(len(f_i), len(f_j))
    phi2 /= (min_d - 1)
    return phi2


In [ ]:
cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
cat_cols

In [ ]:
# Example: assume df is your loaded dataset
# Filter only categorical (object or category) columns
cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

# Calculate ϕ² for each attribute pair
phi2_matrix = pd.DataFrame(index=cat_cols, columns=cat_cols, dtype=float)

for i in cat_cols:
    for j in cat_cols:
        if i == j:
            phi2_matrix.loc[i, j] = 1.0
        else:
            phi2_matrix.loc[i, j] = mean_square_contingency(df, i, j)

phi2_matrix = phi2_matrix.fillna(0)
display(phi2_matrix)


In [ ]:
# Example: group attributes with phi2 > 0.2
threshold = 0.2
groups = []
used = set()

for col in phi2_matrix.columns:
    if col in used:
        continue
    group = [col]
    used.add(col)
    for other_col in phi2_matrix.columns:
        if other_col not in used and phi2_matrix.loc[col, other_col] > threshold:
            group.append(other_col)
            used.add(other_col)
    groups.append(group)

print("Attribute Groups based on φ² similarity:")
for i, g in enumerate(groups):
    print(f"Group {i+1}: {g}")
